<a href="https://colab.research.google.com/github/DajeanArcila/biblioteca_pandas/blob/main/Algoritmo_recomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357260 sha256=7d2abb8bd7db1e906b603fe45303545d691d15b939a764e5551c1db9b2d1fcf1
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


# Ejemplo de Algoritmo de recomendacion


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Cargar el dataset de ejemplo de MovieLens
data = Dataset.load_builtin('ml-100k')

# Usar el algoritmo SVD para el filtrado colaborativo
algo = SVD()

# Evaluar el rendimiento del algoritmo usando validación cruzada
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Entrenar el algoritmo en todo el conjunto de datos
trainset = data.build_full_trainset()
algo.fit(trainset)

# Hacer una predicción para un usuario específico y un ítem
uid = str(196)  # ID del usuario
iid = str(302)  # ID del ítem

pred = algo.predict(uid, iid, verbose=True)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] yes
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9360  0.9387  0.9334  0.9315  0.9371  0.9353  0.0026  
MAE (testset)     0.7398  0.7419  0.7329  0.7364  0.7382  0.7378  0.0030  
Fit time          2.58    2.46    1.73    1.27    1.29    1.86    0.56    
Test time         0.20    0.29    0.12    0.22    0.18    0.20    0.06    
user: 196        item: 302        r_ui = None   est = 4.20   {'was_impossible': False}


##Interpretación
RMSE (Root Mean Squared Error): El RMSE promedio es 0.9375 con una desviación estándar de 0.0046. Esto indica que, en promedio, el error cuadrático medio de las predicciones es aproximadamente 0.9375. Cuanto más bajo sea el RMSE, mejor será el modelo.

MAE (Mean Absolute Error): El MAE promedio es 0.7392 con una desviación estándar de 0.0036. Este valor es similar al RMSE pero es menos sensible a los grandes errores.

Fit time: El tiempo promedio que tomó entrenar el modelo en cada pliegue fue de aproximadamente 1.70 segundos.

Test time: El tiempo promedio que tomó evaluar el modelo en cada pliegue fue de aproximadamente 0.19 segundos.

##Predicción
**User : 196**: ID del usuario para el cual se hizo la predicción.

**item : 302**: ID del ítem para el cual se hizo la predicción.

**r_ui = None**: Valor real de la calificación (no está disponible en este caso).

**est = 4.34**: Estimación de la calificación que el usuario 196 daría al ítem 302. En este caso, se estima que el usuario daría una calificación de 4.34.

**{'was_impossible': False}**: Indica si la predicción fue posible o no. Aquí, fue posible.

# Algoritmo de recomendacion "Case real"

In [2]:
import pandas as pd

# Cargar el dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
data = pd.read_excel(url)

# Mostrar las primeras filas del dataset
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
# Eliminar filas con valores nulos
data = data.dropna(subset=['CustomerID'])

# Convertir CustomerID a string
data['CustomerID'] = data['CustomerID'].astype(str)

# Mostrar las primeras filas del dataset preprocesado
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Crear una matriz de usuario-producto
user_product_matrix = data.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', aggfunc='sum', fill_value=0)

# Mostrar la matriz de usuario-producto
user_product_matrix.head()


StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Convertir la matriz de usuario-producto a un formato compatible con Surprise
reader = Reader(rating_scale=(0, user_product_matrix.values.max()))
data = Dataset.load_from_df(user_product_matrix.stack().reset_index().rename(columns={0: 'rating'}), reader)

# Dividir el dataset en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

# Crear el modelo SVD
algo = SVD()

# Entrenar el modelo
algo.fit(trainset)


In [6]:
from surprise import accuracy

# Evaluar el modelo en el conjunto de prueba
predictions = algo.test(testset)

# Calcular el RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')


RMSE: 12539.7031
RMSE: 12539.703089886561


In [11]:
# Mostrar los primeros 10 IDs de usuario
print(user_product_matrix.index[:10])


Index(['12346.0', '12347.0', '12348.0', '12349.0', '12350.0', '12352.0',
       '12353.0', '12354.0', '12355.0', '12356.0'],
      dtype='object', name='CustomerID')


In [12]:
# Hacer recomendaciones para un usuario específico
user_id = str('12346.0')  # Reemplaza con un ID de usuario existente en tu dataset
user_ratings = user_product_matrix.loc[user_id].values
predictions = [algo.predict(user_id, str(item_id)).est for item_id in user_product_matrix.columns]

# Crear un dataframe con las recomendaciones
recommendations = pd.DataFrame({'StockCode': user_product_matrix.columns, 'PredictedRating': predictions})
recommendations = recommendations.sort_values(by='PredictedRating', ascending=False)

# Mostrar las 10 principales recomendaciones
print(recommendations.head(10))


     StockCode  PredictedRating
0        10002            12540
2447     72134            12540
2449     72586            12540
2450     72598            12540
2451     72709            12540
2452     72741            12540
2453     72780            12540
2454     72783            12540
2455     72811            12540
2456     72812            12540
